<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/WQPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install requests beautifulsoup4 pandas pyarrow tqdm

In [38]:
import os
import re
import time
import csv
import sys
import math
import shutil
import zipfile
import logging
import traceback
from io import StringIO
from urllib.parse import urljoin, urlparse

import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

# ----------------------------
# CONFIG
# ----------------------------
DATASET_PAGE = "https://ihp-wins.unesco.org/dataset/unep-gems-water-global-freshwater-quality-archive"
OUTPUT_DIR = "gems_data"            # where CSVs will be stored
COMBINED_OUT_CSV = "gems_combined.csv"
COMBINED_OUT_PARQUET = "gems_combined.parquet"

# OPTIONAL: to download only some parameters. Leave empty [] to fetch ALL CSV links on the page.
# Match is case-insensitive, applied to link text or file name.
PARAMETERS_FILTER = [
    # Examples (uncomment what you need, or add your own):
    # "pH",
    # "Turbidity",
    # "Electrical Conductance",  # sometimes called "Specific conductance" elsewhere; here it's "Electrical Conductance"
    # "Dissolved Oxygen",
    # "Chloride",
    # "Nitrate",
]

# Maximum number of CSVs to download (0 for unlimited)
MAX_FILES = 0

# Network settings
TIMEOUT_S = 180
RETRIES = 3
BACKOFF_S = 3

# Logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


# ----------------------------
# HELPERS
# ----------------------------
def fetch_html(url, timeout=TIMEOUT_S):
    """Fetch a page and return BeautifulSoup object."""
    logging.info(f"Fetching dataset page: {url}")
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def find_csv_links(soup, base_url):
    """
    Parse the dataset page for CSV links.
    Return list of (text, absolute_url).
    """
    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        # Some links may include params; check file extension conservatively
        if href.lower().endswith(".csv"):
            url_abs = href if href.startswith("http") else urljoin(base_url, href)
            text = a.get_text(strip=True) or os.path.basename(urlparse(url_abs).path)
            links.append((text, url_abs))
    # Deduplicate by URL
    seen = set()
    unique = []
    for t, u in links:
        if u not in seen:
            unique.append((t, u))
            seen.add(u)
    logging.info(f"Found {len(unique)} CSV links on the page.")
    return unique


def matches_filter(text_or_filename, filters):
    """
    Case-insensitive substring match against any filter token.
    If filters empty, always True (no filtering).
    """
    if not filters:
        return True
    s = text_or_filename.lower()
    for f in filters:
        if f.lower() in s:
            return True
    return False


def download_with_retries(url, out_path, retries=RETRIES, backoff=BACKOFF_S, timeout=TIMEOUT_S):
    """Download a file with retries and a progress bar."""
    for attempt in range(1, retries + 1):
        try:
            with requests.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()
                total = int(r.headers.get("Content-Length", 0))
                desc = os.path.basename(out_path) or "download"
                with open(out_path, "wb") as f, tqdm(
                    total=total if total > 0 else None,
                    unit="B",
                    unit_scale=True,
                    desc=f"Downloading {desc}",
                    leave=False,
                ) as pbar:
                    for chunk in r.iter_content(chunk_size=1024 * 1024):
                        if chunk:
                            f.write(chunk)
                            if total > 0:
                                pbar.update(len(chunk))
            return out_path
        except Exception as e:
            logging.warning(f"Attempt {attempt}/{retries} failed for {url}: {e}")
            if attempt == retries:
                raise
            time.sleep(backoff * attempt)


def normalize_columns(df):
    """
    Basic normalization across GEMS parameter CSVs.
    The exact column names in the archive are consistent per resource, but can differ across parameters.
    We try to standardize common fields for ML:
      - timestamp (DateTime or SampleDate)
      - station/site id
      - parameter name
      - value
      - unit
      - lat/lon if present
    """
    df = df.copy()
    # Candidate timestamp columns
    ts_candidates = [
        "DateTime", "Date_Time", "Date", "Datetime", "SampleDate", "SamplingDate",
        "ActivityStartDate", "ActivityDate", "date_time", "sample_datetime", "time"
    ]
    for c in ts_candidates:
        if c in df.columns:
            df["timestamp"] = pd.to_datetime(df[c], errors="coerce", utc=True)
            break
    if "timestamp" not in df.columns:
        # No recognizable timestamp; try to compose if separate date/time present
        date_cols = [c for c in df.columns if "date" in c.lower()]
        time_cols = [c for c in df.columns if "time" in c.lower()]
        if date_cols and time_cols:
            df["timestamp"] = pd.to_datetime(
                df[date_cols[0]].astype(str) + " " + df[time_cols[0]].astype(str),
                errors="coerce", utc=True
            )
        else:
            df["timestamp"] = pd.NaT

    # Station/Location candidates
    station_candidates = [
        "StationID", "StationCode", "MonitoringLocationIdentifier",
        "SiteID", "SiteCode", "Station", "LocationID", "StationIdentifier", "id"
    ]
    for c in station_candidates:
        if c in df.columns:
            df["station_id"] = df[c].astype(str)
            break
    if "station_id" not in df.columns:
        df["station_id"] = None

    # Parameter/Characteristic candidates
    param_candidates = [
        "ParameterName", "CharacteristicName", "Parameter", "Analyte", "Variable"
    ]
    for c in param_candidates:
        if c in df.columns:
            df["parameter"] = df[c].astype(str)
            break
    if "parameter" not in df.columns:
        # Try to infer from file name later
        df["parameter"] = None

    # Value candidates
    value_candidates = [
        "ResultValue", "Value", "Result.MeasureValue", "MeasureValue",
        "ResultMeasureValue", "Result", "ObservationValue"
    ]
    for c in value_candidates:
        if c in df.columns:
            df["value"] = pd.to_numeric(df[c], errors="coerce")
            break
    if "value" not in df.columns:
        df["value"] = pd.NA

    # Unit candidates
    unit_candidates = [
        "Unit", "Units", "MeasureUnit", "ResultUnit", "ResultMeasureUnitCode",
        "UnitName"
    ]
    for c in unit_candidates:
        if c in df.columns:
            df["unit"] = df[c].astype(str)
            break
    if "unit" not in df.columns:
        df["unit"] = None

    # Latitude/Longitude
    lat_candidates = ["Latitude", "LatitudeMeasure", "Lat", "station_latitude"]
    lon_candidates = ["Longitude", "LongitudeMeasure", "Long", "station_longitude"]
    for c in lat_candidates:
        if c in df.columns:
            df["latitude"] = pd.to_numeric(df[c], errors="coerce")
            break
    if "latitude" not in df.columns:
        df["latitude"] = pd.NA
    for c in lon_candidates:
        if c in df.columns:
            df["longitude"] = pd.to_numeric(df[c], errors="coerce")
            break
    if "longitude" not in df.columns:
        df["longitude"] = pd.NA

    # Sort by timestamp if present
    if df["timestamp"].notna().any():
        df = df.sort_values("timestamp")

    # Keep a slim schema first, but preserve original columns at the end for traceability
    cols = ["timestamp", "station_id", "parameter", "value", "unit", "latitude", "longitude"]
    # Append all other columns for audit/tracking
    tail = [c for c in df.columns if c not in cols]
    return df[cols + tail]


def infer_parameter_from_filename(filename):
    name = os.path.splitext(os.path.basename(filename))[0]
    # Clean underscores/dashes and make it human-readable
    name = re.sub(r"[ _\-]+", " ", name).strip()
    return name


# ----------------------------
# MAIN PIPELINE
# ----------------------------
def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Option A: auto-scrape CSVs from the dataset page
    soup = fetch_html(DATASET_PAGE)
    all_csvs = find_csv_links(soup, DATASET_PAGE)

    # Filter by chosen parameters if provided
    filtered = []
    for text, url in all_csvs:
        # Check link text and filename
        fname = os.path.basename(urlparse(url).path)
        if matches_filter(text, PARAMETERS_FILTER) or matches_filter(fname, PARAMETERS_FILTER):
            filtered.append((text, url))
    # If no filter chosen, download everything found
    if PARAMETERS_FILTER and not filtered:
        logging.warning("No links matched your PARAMETERS_FILTER. Falling back to ALL CSVs.")
        filtered = all_csvs
    if not PARAMETERS_FILTER:
        filtered = all_csvs

    # Respect MAX_FILES if set
    if MAX_FILES and len(filtered) > MAX_FILES:
        filtered = filtered[:MAX_FILES]

    logging.info(f"{len(filtered)} CSV(s) selected for download.")

    downloaded_files = []
    for (text, url) in filtered:
        filename = os.path.basename(urlparse(url).path)
        if not filename.lower().endswith(".csv") or filename == "":
            # Final fallback to a sensible name
            filename = re.sub(r"\\W+", "_", text or "data") + ".csv"
        out_path = os.path.join(OUTPUT_DIR, filename)
        if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
            logging.info(f"Already downloaded: {filename}")
        else:
            logging.info(f"Downloading: {text} -> {filename}")
            try:
                download_with_retries(url, out_path)
            except Exception:
                logging.error(f"Failed to download {url}\n{traceback.format_exc()}")
                continue
        downloaded_files.append(out_path)

    if not downloaded_files:
        logging.error("No CSVs were downloaded. Check the dataset page or your filters.")
        sys.exit(1)

    # Load & normalize, also infer parameter from file if missing
    frames = []
    for fp in downloaded_files:
        try:
            df = pd.read_csv(fp)
            df = normalize_columns(df)
            # If parameter column is empty, infer from filename once
            if df["parameter"].isna().all() or (df["parameter"].astype(str) == "None").all():
                df["parameter"] = infer_parameter_from_filename(fp)
            frames.append(df)
            logging.info(f"Loaded {os.path.basename(fp)} — rows: {len(df)}")
        except Exception:
            logging.error(f"Error reading {fp}\n{traceback.format_exc()}")

    if not frames:
        logging.error("No data frames were created from the downloaded CSVs.")
        sys.exit(1)

    combined = pd.concat(frames, ignore_index=True)

    # Optional: basic quality filters for ML
    # Drop rows with no value or timestamp
    before = len(combined)
    # combined = combined.dropna(subset=["value"])
    # Keep timestamp if present; you can relax this if needed
    # if "timestamp" in combined.columns:
    #     combined = combined.dropna(subset=["timestamp"])

    logging.info(f"Combined rows before cleanup: {before} | after cleanup: {len(combined)}")

    # Save outputs
    combined.to_csv(COMBINED_OUT_CSV, index=False)
    logging.info(f"Saved combined CSV -> {COMBINED_OUT_CSV}")

    # Parquet (smaller & faster for ML)
    try:
        combined.to_parquet(COMBINED_OUT_PARQUET, index=False)
        logging.info(f"Saved combined Parquet -> {COMBINED_OUT_PARQUET}")
    except Exception as e:
        logging.warning(f"Parquet save failed (install pyarrow): {e}")

    # Quick preview
    print("\n=== Preview (first 10 rows) ===")
    print(combined.head(10))


if __name__ == "__main__":
    main()

/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-1153237744.py:300: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)
/tmp/ipython-input-11532


=== Preview (first 10 rows) ===
                  timestamp station_id             parameter     value  unit  \
0 1963-08-01 00:00:00+00:00   IND02067  processed alkalinity   96.2541  mg/l   
1 1963-08-01 00:00:00+00:00   IND02067  processed alkalinity    4.9800  mg/l   
2 1963-09-01 00:00:00+00:00   IND02067  processed alkalinity    2.4900  mg/l   
3 1963-09-01 00:00:00+00:00   IND02067  processed alkalinity   90.0000  mg/l   
4 1963-10-01 00:00:00+00:00   IND02067  processed alkalinity  162.9100  mg/l   
5 1963-10-01 00:00:00+00:00   IND02067  processed alkalinity   24.9000  mg/l   
6 1963-11-01 00:00:00+00:00   IND02067  processed alkalinity  122.4430  mg/l   
7 1963-11-01 00:00:00+00:00   IND02067  processed alkalinity    2.4900  mg/l   
8 1963-11-09 00:00:00+00:00   IND02225  processed alkalinity    0.0000  mg/l   
9 1963-11-09 00:00:00+00:00   IND02225  processed alkalinity  144.2620  mg/l   

    latitude  longitude   Latitude  Longitude              time        id  \
0  25.330

In [39]:
import pandas as pd

df = pd.read_csv("gems_combined.csv")
print(df.info())
print(df['parameter'].value_counts().head(20))
print(df['station_id'].nunique(), "stations")

/tmp/ipython-input-3252709269.py:3: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("gems_combined.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560953 entries, 0 to 7560952
Data columns (total 18 columns):
 #   Column                Dtype  
---  ------                -----  
 0   timestamp             object 
 1   station_id            object 
 2   parameter             object 
 3   value                 float64
 4   unit                  object 
 5   latitude              float64
 6   longitude             float64
 7   Latitude              float64
 8   Longitude             float64
 9   time                  object 
 10  id                    object 
 11  Depth                 float64
 12  Parameter Code        object 
 13  Analysis Method Code  object 
 14  Value Flags           object 
 15  Value                 float64
 16  Unit                  object 
 17  Data Quality          object 
dtypes: float64(7), object(11)
memory usage: 1.0+ GB
None
parameter
processed dissolved gas             713528
processed carbon                    566817
processed electrical conductance 

In [40]:
df = df.dropna(subset=['timestamp', 'value'])
df = df[df['value'] >= 0]

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

# Example: resample to daily averages
daily = df.set_index('timestamp').groupby('parameter')['value'].resample('1D').mean().reset_index()

In [41]:
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month
df['dayofweek'] = df['timestamp'].dt.dayofweek

In [42]:
df['turbidity_lag1'] = df['value'].shift(1)
df['turbidity_lag2'] = df['value'].shift(2)
df['turbidity_lag24'] = df['value'].shift(24)


In [43]:
df['turbidity_roll_mean_3'] = df['value'].rolling(3).mean()
df['turbidity_roll_std_3'] = df['value'].rolling(3).std()
df['turbidity_roll_mean_24'] = df['value'].rolling(24).mean()


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Drop rows with NaNs from lag/rolling features
data = df.dropna()

X = data[['hour','day','month','dayofweek',
          'turbidity_lag1','turbidity_lag24',
          'turbidity_roll_mean_3','turbidity_roll_mean_24']]
y = data['value']   # target = future turbidity

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestRegressor(n_estimators=300)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))

MAE: 6.872360895472459


In [33]:
import pandas as pd

df_loaded = pd.read_csv('gems_combined.csv')
print("DataFrame Head:")
display(df_loaded.head())
print("\nDataFrame Info:")
df_loaded.info()

DataFrame Head:


,timestamp,station_id,parameter,value,unit,latitude,longitude,Latitude,Longitude,time,id,Depth,Parameter Code,Analysis Method Code,Value Flags,Value,Unit,Data Quality



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   timestamp             0 non-null      object
 1   station_id            0 non-null      object
 2   parameter             0 non-null      object
 3   value                 0 non-null      object
 4   unit                  0 non-null      object
 5   latitude              0 non-null      object
 6   longitude             0 non-null      object
 7   Latitude              0 non-null      object
 8   Longitude             0 non-null      object
 9   time                  0 non-null      object
 10  id                    0 non-null      object
 11  Depth                 0 non-null      object
 12  Parameter Code        0 non-null      object
 13  Analysis Method Code  0 non-null      object
 14  Value Flags           0 non-null      object
 15  Value                 0 non-null   

In [45]:
import joblib
joblib.dump(model, "turbidity_model.pkl")


['turbidity_model.pkl']

In [47]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.6 MB/s eta 0:00:00


In [70]:
import streamlit as st
import joblib
import pandas as pd

model = joblib.load("turbidity_model.pkl")

st.title("AI Water Quality Predictor")

uploaded = st.file_uploader("Upload latest water data CSV")

if uploaded:
    df = pd.read_csv(uploaded)
    # preprocess same as training...
    pred = model.predict(df)
    st.line_chart(pred)


2026-01-25 03:59:36.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.770 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 03:59:36.781 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Please replace `YOUR_NGROK_AUTH_TOKEN` with your actual ngrok authtoken from [https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken) and run the cell below.

In [72]:
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual authtoken
ngrok.set_auth_token("38jUXztDRrcg7NDyuJGe8YsZWiW_7nkbtMVLdK1LS6ScbktRr")

In [87]:
from pyngrok import ngrok
import subprocess
import os
import time

# Terminate any existing ngrok tunnels
ngrok.kill()

# Start a Streamlit app in the background and capture its output
# Redirect stdout/stderr to a file to prevent Colab from hanging
log_file = open("streamlit_log.txt", "w")
streamlit_process = subprocess.Popen(["streamlit", "run", "app.py"],
                                     stdout=log_file, stderr=log_file,
                                     env=os.environ.copy())

print("Streamlit app started in background. Waiting for it to become available...")
time.sleep(5) # Give Streamlit some time to start

# Get the public URL
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit App URL: {public_url}")
except Exception as e:
    print(f"Failed to establish ngrok tunnel: {e}")
    print("Check 'streamlit_log.txt' for Streamlit app errors if the above steps fail.")

Streamlit app started in background. Waiting for it to become available...
Streamlit App URL: NgrokTunnel: "https://acrogenic-nontransitionally-shin.ngrok-free.dev" -> "http://localhost:8501"


In [77]:
!tail -n 200 ~/.streamlit/logs/* 2>/dev/null || true

In [78]:
!tail -n 200 streamlit_log.txt




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8508
  Network URL: http://172.28.0.12:8508
  External URL: http://34.186.111.12:8508

